In [1]:
import pandas as pd
import json

C:\Users\jaiga\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\jaiga\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [33]:
file_path = 'B4_UDS4_programmatic_crosswalk.xlsx'
uds3_rdcp = pd.read_excel(file_path,sheet_name='B4 UDS3 REDCap')
uds4_rdcp = pd.read_excel(file_path,sheet_name='B4 UDS4 REDCap')
df = pd.read_excel(file_path,sheet_name='Template Mappings REDCap')

In [34]:
uds4_rdcp.rename(columns = {"UDS4 data element name":"UDS4 data element"},inplace=True)

In [35]:
uds4_rdcp = uds4_rdcp.iloc[:, :12]
uds4_rdcp.head(5)

,UDS4 data element,Nearest UDS3 variable,New in UDS4,UDS3 Mappable,Change in form,Positional change,Question label change,Data type change,Conformity change,Change in response LEVELS,Change in response LABELS,Special mapping
0,PTID__B4,PTID__B4,No,Yes,No,No,Yes,No,No,No,NaN,NaN
1,ADCID__B4,ADCID__B4,No,Yes,No,Yes,Yes,No,Yes,No,Yes,NaN
2,VISITNUM__B4,VISITNUM__B4,No,Yes,No,Yes,Yes,No,NaN,NaN,No,NaN
3,PACKET__B4,NaN,UDS3 DED Only,UDS3 DED Only,No,Yes,Yes,No,No,No,No,NaN
4,FORMVER__B4,FORMVER__B4,No,Yes,No,Yes,Yes,No,Yes,No,No,NaN


In [36]:
test = pd.merge(df,uds4_rdcp, on='UDS4 data element')

In [37]:
def classify_variable(df):

    def classify(group):
        if group['Special mapping'].eq('Structured').any():
            return 'sp'
        elif group['Special mapping'].isna().all():
            if group['Conformity change'].eq('Yes').any() or group['Data type change'].eq('Yes').any() or group['Change in form'].eq('Yes').any():
                return 'cc'
            elif group['UDS4 data element'].iloc[0] == group['UDS3 data element'].iloc[0]:
                return 'direct'
            else:
                
                return 'cc'
        else:
            return 'complex'

    # Group by 'UDS4 data element' and classify each group
    classifications = df.groupby('UDS4 data element').apply(classify)

    # Map the classifications back to the original DataFrame
    df = df.merge(classifications.rename('class'), on='UDS4 data element', how='left')

    return df


In [38]:
result = classify_variable(test)

C:\Users\jaiga\AppData\Local\Temp\ipykernel_22900\3649334300.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  classifications = df.groupby('UDS4 data element').apply(classify)


In [39]:
result.tail(5)

,UDS4 data element,UDS3 data element,Mapping type,UDS3 value,UDS4 value,Reversible to UDS3,Identical mapping,NOTES AND DISCUSSION POINTS,Nearest UDS3 variable,New in UDS4,UDS3 Mappable,Change in form,Positional change,Question label change,Data type change,Conformity change,Change in response LEVELS,Change in response LABELS,Special mapping,class
274,CDRLANG,CDRLANG,Response LABELS,<b>None - 0 </b><br/>No language difficulty or...,<b>None = 0 </b><br/>No language difficulty or...,Yes,NaN,NaN,CDRLANG,No,Partial,No,No,Yes,No,Yes,No,Yes,NaN,cc
275,CDRLANG,CDRLANG,Response LABELS,<b>Questionable - 0.5</b><br>Consistent mild w...,<b>Questionable = 0.5</b><br>Consistent mild w...,Yes,NaN,NaN,CDRLANG,No,Partial,No,No,Yes,No,Yes,No,Yes,NaN,cc
276,CDRLANG,CDRLANG,Response LABELS,<b>Mild - 1</b><br/>Moderate word finding diff...,<b>Mild = 1</b><br/>Moderate word-finding diff...,Yes,NaN,NaN,CDRLANG,No,Partial,No,No,Yes,No,Yes,No,Yes,NaN,cc
277,CDRLANG,CDRLANG,Response LABELS,<b>Moderate - 2</b><br/>Moderate to severe imp...,<b>Moderate = 2</b><br/>Moderate to severe imp...,Yes,NaN,NaN,CDRLANG,No,Partial,No,No,Yes,No,Yes,No,Yes,NaN,cc
278,CDRLANG,CDRLANG,Response LABELS,<b>Severe - 3</b><br/>Severe comprehension def...,<b>Severe = 3</b><br/>Severe comprehension def...,Yes,NaN,NaN,CDRLANG,No,Partial,No,No,Yes,No,Yes,No,Yes,NaN,cc


In [40]:
result['class'].value_counts()

class
cc         224
sp          23
direct      21
complex     11
Name: count, dtype: int64

In [41]:
result.to_csv("testing.csv", index=False)

In [42]:
result['NOTES AND DISCUSSION POINTS'] = result['NOTES AND DISCUSSION POINTS'].fillna("NA")
result['Reversible to UDS3'] = result['Reversible to UDS3'].fillna("NA")

In [43]:
def categorize_variables(df):
    
    direct_mappings = []
    conditional_consistency_mappings = []
    structured_transformation_mappings = []
    complex_mappings = []

    # Group by 'UDS3 data element' to process holistically
    for uds3_var, group in df.groupby('UDS3 data element'):
        # Get the class for the group
        variable_class = group['class'].iloc[0]

        # Collecting UDS4 variables and crosswalk mappings
        uds4_vars = group['UDS4 data element'].unique()
        crosswalk_mappings = [
            {
                "mapping_type": row["Mapping type"],
                "mappings": [
                    {
                        "UDS3_value": row["UDS3 value"],
                        "UDS4_value": row["UDS4 value"],
                        "reversible": row["Reversible to UDS3"],
                        "note": row.get("NOTES AND DISCUSSION POINTS", ""),
                    }
                ],
            }
            for _, row in group.iterrows()
        ]

        # Assign to the appropriate category based on the class
        if variable_class == "sp":
            structured_transformation_mappings.append({
                "UDS3_variable": uds3_var,
                "UDS4_variable": group['UDS4 data element'].iloc[0],
                "crosswalk_mappings": crosswalk_mappings,
            })
        elif variable_class == "cc":
            conditional_consistency_mappings.append({
                "UDS3_variable": uds3_var,
                "UDS4_variable": group['UDS4 data element'].iloc[0],
                "crosswalk_mappings": crosswalk_mappings,
            })
            
        elif variable_class == "complex":
            complex_mappings.append({
                "UDS3_variable": uds3_var,
                "UDS4_variable": group['UDS4 data element'].iloc[0],
                "crosswalk_mappings": crosswalk_mappings,
            })
        
        elif variable_class == "direct":
            direct_mappings.append({
                "UDS3_variable": uds3_var,
                "UDS4_variable": group['UDS4 data element'].iloc[0],
                "crosswalk_remappings": crosswalk_mappings,
            })

    # Combine all mappings into a single JSON structure
    all_mappings = {
        "Direct_Mappings": direct_mappings,
        "Conditional_Consistency": conditional_consistency_mappings,
        "Structured_Transformations": structured_transformation_mappings,
        "High Complexity" : complex_mappings,
    }
    return all_mappings


In [44]:
all_mappings = categorize_variables(result)

output_path = 'Final_B4.json'
with open(output_path, 'w') as json_file:
    json.dump(all_mappings, json_file, ensure_ascii=False, indent=4)

print(f"All mappings JSON saved to: {output_path}")

All mappings JSON saved to: Final_B4.json
